In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv("Data/fake reviews dataset.csv")

In [2]:
# Clean up the category names
old_cat = ['Home_and_Kitchen_5', 'Sports_and_Outdoors_5', 'Electronics_5',
       'Movies_and_TV_5', 'Tools_and_Home_Improvement_5',
       'Pet_Supplies_5', 'Kindle_Store_5', 'Books_5', 'Toys_and_Games_5',
       'Clothing_Shoes_and_Jewelry_5']
new_cat = ['Home and Kitchen', 'Sports and Outdoors', 'Electronics',
        'Movies and TV', 'Tools and Home Improvement',
        'Pet Supplies', 'Kindle Store', 'Books', 'Toys and Games',
        'Clothing Shoes and Jewelry']
df['category'].replace(to_replace=old_cat, value=new_cat, inplace=True)

# Remove any duplicate rows, then check for NA values for each column
df = df.drop_duplicates(subset='text_')
df['text_'].replace(".   .                   ", np.nan, inplace=True)
df.dropna(subset=['text_'], inplace=True)

print(df.isnull().sum())

# Rename columns so that they are more descriptive
colnames_dict = {"label": "type", "text_": "comments"}
df.rename(columns=colnames_dict, inplace=True)

# Map the type column to numeric/boolean values:
  # OR (Original reviews, presumably human-created and authentic) = 0
  # CG (Computer-generated fake reviews) = 1

type_dict_map = {'OR': 0 ,'CG':1}
df['type'] = df['type'].map(type_dict_map)
df.head()

category    0
rating      0
label       0
text_       0
dtype: int64


,category,rating,type,comments
0,Home and Kitchen,5.0,1,"Love this! Well made, sturdy, and very comfor..."
1,Home and Kitchen,5.0,1,"love it, a great upgrade from the original. I..."
2,Home and Kitchen,5.0,1,This pillow saved my back. I love the look and...
3,Home and Kitchen,1.0,1,"Missing information on how to use it, but it i..."
4,Home and Kitchen,5.0,1,Very nice set. Good quality. We have had the s...


In [18]:

target_var  = 'type'
feature_var = [ 'rating', 'comments']
X = df[feature_var]
y = df[target_var]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=101)
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

print("X_train shape:", X_train.shape)
print(X_train.head())
print("X_test shape:", X_test.shape)
print(X_test.head())
print("y_train shape:", y_train.shape)
print(y_train[:5])
print("y_test shape:", y_test.shape)
print(y_test[:5])

X_train shape: (28287, 2)
       rating                                           comments
7569      5.0  Great Cooler. Cheaper than other high end cool...
30086     4.0  Wow i can't believe Richards left me hanging n...
28669     2.0  Just didn't hold my interest. I finished it, b...
28491     4.0  it's a tough subject, this book has the potent...
3986      4.0  This is a decent storage rack. The only proble...
X_test shape: (12124, 2)
       rating                                           comments
27462     2.0  'Name that Doggy!' Because it was an easy read...
4476      5.0  These are good targets to add a bit of realism...
38652     5.0  I love this watch!  It has the wide band which...
11314     3.0  It does as advertised, mostly. It's on the bug...
21592     5.0  I have 2 boxers that I felt needed a little wa...
y_train shape: (28287, 1)
[[0]
 [0]
 [0]
 [1]
 [1]]
y_test shape: (12124, 1)
[[1]
 [0]
 [1]
 [0]
 [0]]


In [11]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a pipeline to vectorize the text data and then apply the logistic regression model
pipeline_log = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000)),
    ('logistic', LogisticRegression())
])

# Transform comment to tfidf
X_train_tfidf = pipeline_log.named_steps['tfidf'].fit_transform(X_train['comments'])
X_test_tfidf = pipeline_log.named_steps['tfidf'].fit_transform(X_test['comments'])

print(X_train_tfidf[0])
print(X_test_tfidf[0])

  (0, 18892)	0.07557085908011478
  (0, 8448)	0.07429557043831958
  (0, 10118)	0.060008654438684715
  (0, 18161)	0.04731021720574824
  (0, 6107)	0.0814416734565174
  (0, 1246)	0.07435528176479633
  (0, 10520)	0.10955999856061753
  (0, 7465)	0.09871763912247238
  (0, 19512)	0.048734521557744916
  (0, 3088)	0.08041327408864679
  (0, 5945)	0.07260836411962672
  (0, 19584)	0.07736553529731821
  (0, 6251)	0.05900520552150097
  (0, 10382)	0.05957489879646748
  (0, 9767)	0.03990911378434627
  (0, 1043)	0.0457694410354829
  (0, 15564)	0.09274967045411944
  (0, 1694)	0.05922830272487444
  (0, 9824)	0.05536249329485334
  (0, 5943)	0.08208495123545609
  (0, 18263)	0.14941930686932034
  (0, 11745)	0.05841380157258264
  (0, 12242)	0.025317707409692938
  (0, 18468)	0.11412511367764698
  (0, 9039)	0.05735535990164553
  :	:
  (0, 17967)	0.1931828085130935
  (0, 19472)	0.07889929009013393
  (0, 1143)	0.09554695741631528
  (0, 10136)	0.13324902837598637
  (0, 12341)	0.1074319332195513
  (0, 18100)	0.1558

In [22]:
# Logistic Regression with 'comments' column
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import metrics 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a pipeline to vectorize the text data and then apply the logistic regression model
pipeline_log = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000)),
    ('logistic', LogisticRegression())
])

# Perform cross-validation and print the scores
cv_scores_log = cross_val_score(pipeline_log, X_train['comments'], y_train.ravel(), cv=5)  # Choose appropriate cv value
print("Cross-Validation Scores:", cv_scores_log)

# Fit pipeline to training data and make predictions
pipeline_log.fit(X_train['comments'], y_train.ravel())
predictions_log = pipeline_log.predict(X_test['comments'])

Cross-Validation Scores: [0.90261577 0.90509014 0.89711861 0.8934064  0.89676507]


In [23]:
## Print model evaluation metrics
# Calculate accuracy
acc_log = metrics.accuracy_score(predictions_log, y_test.ravel())
print('Accuracy of logistic regression model:', acc_log)

# Print classification report and confusion matrix
print(metrics.classification_report(y_test, predictions_log))
print(metrics.confusion_matrix(y_test, predictions_log))

The accuracy of logistic regression model is 0.8992081821181128
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      6042
           1       0.91      0.88      0.90      6082

    accuracy                           0.90     12124
   macro avg       0.90      0.90      0.90     12124
weighted avg       0.90      0.90      0.90     12124

[[5530  512]
 [ 710 5372]]


In [27]:
# Logistic Regression with 'comments' and 'rating' columns
from sklearn.compose import ColumnTransformer

# Create a pipeline for TF-IDF vectorization of the 'comments' column
tfidf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000))
])

# Create a ColumnTransformer to apply different transformations to different columns
preprocessor_log = ColumnTransformer(
    transformers=[
        ('comments_tfidf', tfidf_pipeline, 'comments'),  # Apply TF-IDF to 'comments' column
        ('passthrough', 'passthrough', ['rating'])  # Pass through 'type' column as it is
    ],
    remainder='drop'  # Drop any columns not specified in the transformers
)

# Create the final pipeline by combining preprocessor and logistic regression
pipeline_log2 = Pipeline([
    ('preprocessor', preprocessor_log),
    ('logistic', LogisticRegression(solver='sag'))
])

# Perform cross-validation and print the scores
cv_scores_log2 = cross_val_score(pipeline_log2, X_train, y_train.ravel(), cv=5)
print("Cross-Validation Scores:", cv_scores_log2)

# Fit pipeline to training data and make predictions
pipeline_log2.fit(X_train, y_train.ravel())
predictions_log2 = pipeline_log2.predict(X_test)


Cross-Validation Scores: [0.90296925 0.90544362 0.89623475 0.89429026 0.89552766]


In [28]:
## Print model evaluation metrics
# Calculate accuracy
acc_log2 = metrics.accuracy_score(predictions_log2, y_test.ravel())
print('Accuracy of logistic regression model:', acc_log2)

# Print classification report and confusion matrix
print(metrics.classification_report(y_test, predictions_log2))
print(metrics.confusion_matrix(y_test, predictions_log2))

Accuracy of logistic regression model: 0.8992081821181128
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      6042
           1       0.91      0.88      0.90      6082

    accuracy                           0.90     12124
   macro avg       0.90      0.90      0.90     12124
weighted avg       0.90      0.90      0.90     12124

[[5530  512]
 [ 710 5372]]


In [20]:
# Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a pipeline to vectorize the text data and then apply the logistic regression model
pipeline_multiNB = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000)),
    ('Multinomial NB', MultinomialNB())
])

# Perform cross-validation and print the scores
cv_score_multiNB = cross_val_score(pipeline_multiNB, X_train['comments'], y_train.ravel(), cv=5)  # Choose appropriate cv value
print("Cross-Validation Scores:", cv_score_multiNB)

# Fit pipeline to training data and make predictions
pipeline_multiNB.fit(X_train['comments'], y_train.ravel())
predictions_multiNB = pipeline_multiNB.predict(X_test['comments'])

# Print classification report and confusion matrix
print(classification_report(y_test, predictions_multiNB))
print(confusion_matrix(y_test, predictions_multiNB))

Cross-Validation Scores: [0.86002121 0.86532344 0.85928938 0.85610748 0.86070355]
              precision    recall  f1-score   support

           0       0.92      0.80      0.86      6042
           1       0.83      0.93      0.88      6082

    accuracy                           0.87     12124
   macro avg       0.87      0.87      0.87     12124
weighted avg       0.87      0.87      0.87     12124

[[4861 1181]
 [ 425 5657]]


In [32]:
# Multinomial NB with 'comments' and 'rating' columns
from sklearn.compose import ColumnTransformer

# Create a ColumnTransformer to apply different transformations to different columns
preprocessor_log = ColumnTransformer(
    transformers=[
        ('comments_tfidf', tfidf_pipeline, 'comments'),  # Apply TF-IDF to 'comments' column
        ('passthrough', 'passthrough', ['rating'])  # Pass through 'type' column as it is
    ],
    remainder='drop'  # Drop any columns not specified in the transformers
)

# Create the final pipeline by combining preprocessor and logistic regression
pipeline_multiNB2 = Pipeline([
    ('preprocessor', preprocessor_log),
    ('Multinomial NB', MultinomialNB())
])

# Perform cross-validation and print the scores
cv_scores_multiNB2 = cross_val_score(pipeline_multiNB2, X_train, y_train.ravel(), cv=5)
print("Cross-Validation Scores:", cv_scores_multiNB2)

# Fit pipeline to training data and make predictions
pipeline_multiNB2.fit(X_train, y_train.ravel())
predictions_multiNB2 = pipeline_multiNB2.predict(X_test)

# Print classification report and confusion matrix
print(classification_report(y_test, predictions_multiNB))
print(confusion_matrix(y_test, predictions_multiNB))


Cross-Validation Scores: [0.84446801 0.85295157 0.84620824 0.84373343 0.85204172]
              precision    recall  f1-score   support

           0       0.92      0.80      0.86      6042
           1       0.83      0.93      0.88      6082

    accuracy                           0.87     12124
   macro avg       0.87      0.87      0.87     12124
weighted avg       0.87      0.87      0.87     12124

[[4861 1181]
 [ 425 5657]]


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


# Define the pipeline
pipeline_nb = Pipeline([
    # Remove the max_features limitation at the end.
    ('tfidf', TfidfVectorizer(max_features=20000)),  # TF-IDF vectorizer
    ('classifier', GaussianNB())    # Gaussian Naive Bayes classifier
])

# Transform comment to tfidf
X_train_tfidf = pipeline_nb.named_steps['tfidf'].fit_transform(X_train['comments'])
X_test_tfidf = pipeline_nb.named_steps['tfidf'].fit_transform(X_test['comments'])

X_train_tfidf = X_train_tfidf.toarray()
X_test_tfidf = X_test_tfidf.toarray()


kfold = KFold(n_splits=5, shuffle=False, random_state=None)
rkfold = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None)

params = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5] }
model = GaussianNB()

grid_nb = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=5, verbose=1)

# Fit the function to train set 
grid_nb.fit(X_train_tfidf, y_train)

# Find the best parameter and see how well it performs on test set
print(grid_nb.best_params_)
print(grid_nb.score(X_test_tfidf, y_test))
print(grid_nb.cv_results_['mean_test_score'])

model_nb = pipeline_nb.named_steps['classifier'].fit(X_train_tfidf, y_train.ravel())

# Evaluate the model
train_accuracy = pipeline_nb.named_steps['classifier'].score(X_train_tfidf, y_train)
test_accuracy = pipeline_nb.named_steps['classifier'].score(X_test_tfidf, y_test)

feature_names =  pipeline_nb.named_steps['tfidf'].get_feature_names_out()
num_features = len(feature_names)

print("Number of features (vocabulary size):", num_features)
print("This accuracy score shows")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

predict_nb = model_nb.predict(X_test_tfidf)

acc = accuracy_score(predict_nb, y_test)
precision = precision_score(predict_nb, y_test)
recall = recall_score(predict_nb, y_test)
f1 = f1_score(predict_nb, y_test)

print('The accuracy score of NB is', acc)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


# Below are code for validation
# print('kFold accuracy scores are', cross_val_score(pipeline_nb.named_steps['classifier'], X_train_tfidf, y_train, scoring='precision', cv= kfold, n_jobs=-1))
# print('repeated kFold accuracy scores are', cross_val_score(pipeline_nb.named_steps['classifier'], X_train_tfidf, y_train, scoring='precision', cv= rkfold, n_jobs=-1))

# print('The average KFold scores is', np.mean(cross_val_score(pipeline.named_steps['classifier'], X_train_tfidf, y_train, scoring='accuracy', cv=kfold, n_jobs=-1)))
# print('The average RepeatedKFold score is', np.mean(cross_val_score(pipeline.named_steps['classifier'], X_train_tfidf, y_train, cv=rkfold, n_jobs=-1)))

Fitting 5 folds for each of 5 candidates, totalling 25 fits


c:\Users\Chris\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Chris\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Chris\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Chris\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

{'var_smoothing': 1e-05}
0.49843286044209834
[0.74458255 0.74546634 0.74642075 0.74825907 0.75020343]
Number of features (vocabulary size): 20000
This accuracy score shows
Training Accuracy: 0.8444868667585816
Testing Accuracy: 0.49843286044209834
The accuracy score of NB is 0.49843286044209834
Precision: 0.0024662939822426835
Recall: 0.5172413793103449
F1 Score: 0.0049091801669121264


In [5]:
from sklearn.metrics import accuracy_score


# Ask for user input
rating = int(input("Enter the rating (1-5): "))
comment = input("Enter the comment: ")

# Create a DataFrame with the user input
user_data = pd.DataFrame({'rating': [rating],
                          'comments': [comment]})

# Transform the user input using the TF-IDF vectorizer
user_data_tfidf = pipeline.named_steps['tfidf'].transform(user_data['comments'])

# Predict the output using the trained classifier
prediction = pipeline.named_steps['classifier'].predict(user_data_tfidf.toarray())

# Calculate accuracy if actual output is available
# OR (Original reviews, presumably human-created and authentic) = 0
# CG (Computer-generated fake reviews) = 1
actual_output = 0  
accuracy = accuracy_score([actual_output], prediction)
# Print the prediction and accuracy
if prediction[0] == 0:
    print("Original review (OR)")
else:
    print("Computer-generated fake review (CG)")

print("Prediction Accuracy:", accuracy)

ValueError: invalid literal for int() with base 10: ''

<br>
<br>
<br>
<br>
<br>
Below are backup code that didn't really work


In [ ]:
#Transform each comment row into tfidf array
df['comments'] = df['comments'].astype(str)

vectorizer = TfidfVectorizer()
# print(df['comments'])
for index, row in df.iterrows():
    # Transform the comment into a TF-IDF array
    comment = row['comments']
    # Check if the comment is empty or contains only stop words
    

    # Transform the comment into a TF-IDF array
    comment_tfidf = vectorizer.fit_transform([comment]).toarray()
    # Replace the original comment with its TF-IDF array
    df.at[index, 'comments'] = comment_tfidf

# response = vectorizer.fit_transform(df['comments'])
# df['comments'] = response.getnnz()


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline

# Initialize the Naive Bayes classifier
model = GaussianNB()
print("error1")
# Train the classifier
model.fit(X_train, y_train)
print("error2")

# Predictions on the training set
train_preds = model.predict(X_train)
print("error3")

# Predictions on the testing set
test_preds = model.predict(X_test)

# Evaluate the model
train_accuracy = accuracy_score(y_train, train_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)
